In [1]:
import pandas
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text

In [2]:
# load dataset
dataframe = pandas.read_csv("tet.csv", header=0)
dataframe.drop_duplicates(inplace = True)
data= dataframe
print(dataframe.head())

print(type(data['text']))

                                                text    class
0  ﻿ اعتقل اندونيسي من قبل الشرطة بعد ان اثار جدل...  religon
1  ﻿ عمان - الراي - اكد المهندس نضال الحديد امين ...      env
2  (2) امه هي امنه بنت وهب بن عبد مناف بن زهره بن...  religon
3  (ناسا) لتركيب نظام قياسي ضوئي شمسي طيفي لرصد ا...       ST
4  @ 3drees: حماقي حلف بالله انه ما يستمتع الا مع...      art
<class 'pandas.core.series.Series'>


In [3]:
train_size = int(len(data) * .8)

print(int(len(data['text'])))
print(train_size)

2393
1914


In [4]:
texts= data['text']
tags = data['class']


train_posts = data['text'][:train_size]
train_tags = data['class'][:train_size]



test_posts = data['text'][train_size:]
test_tags =  data['class'][train_size:]

In [5]:
tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

x_train = tokenizer.texts_to_matrix(train_posts, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_posts, mode='tfidf')


In [37]:
encoder = LabelEncoder()
encoder.fit(tags)
tagst=encoder.fit_transform(tags)

num_classes = int((len(set(tagst))))
print((len(set(tagst))))

y_train = encoder.fit_transform(train_tags)
y_test = encoder.fit_transform(test_tags)



6


In [38]:
y_train= keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


num_labels = int(len(y_train.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size


In [42]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives


In [43]:

# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [44]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])


In [45]:
batch_size = 100
epochs = 2

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h1')


Epoch 1/2
18/18 [==============================] - 6s 276ms/step - loss: 0.9842 - categorical_accuracy: 0.6653 - recall: 0.8066 - precision: 0.3348 - f1_metric: 0.4723 - true_positives: 862.8421 - true_negatives: 3422.4737 - false_positives: 1510.1579 - false_negatives: 123.6842 - val_loss: 0.2724 - val_categorical_accuracy: 0.9167 - val_recall: 0.9740 - val_precision: 0.4022 - val_f1_metric: 0.5724 - val_true_positives: 187.0000 - val_true_negatives: 682.0000 - val_false_positives: 278.0000 - val_false_negatives: 5.0000
Epoch 2/2
18/18 [==============================] - 4s 231ms/step - loss: 0.0179 - categorical_accuracy: 0.9987 - recall: 0.9987 - precision: 0.6085 - f1_metric: 0.7571 - true_positives: 985.5789 - true_negatives: 4332.9474 - false_positives: 599.6842 - false_negatives: 0.9474 - val_loss: 0.1445 - val_categorical_accuracy: 0.9635 - val_recall: 0.9896 - val_precision: 0.4922 - val_f1_metric: 0.6634 - val_true_positives: 190.0000 - val_true_negatives: 764.0000 - val_false

In [46]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [47]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(x_test,y_test,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)


Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[4.277341365814209, 0.5344467759132385, 0.705636739730835, 0.2680412232875824, 0.39802804589271545, 338.0, 1472.0, 923.0, 141.0]


In [49]:
for x in data["text"][:25]:

    tokens = tokenizer.texts_to_matrix([x], mode='tfidf')

    c=model.predict(np.array(tokens))
    cc=model.predict_classes(tokens)
    xc = encoder.inverse_transform(cc)


    print(c,"= \t",cc,"\t",xc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[0.36428607 0.24974772 0.04651052 0.2054654  0.52084166 0.993569  ]] = 	 [5] 	 ['religon']
[[0.3030743  0.46091425 0.80617106 0.24373052 0.38295197 0.61700827]] = 	 [2] 	 ['env']
[[0.440311   0.21136141 0.02192056 0.15663287 0.3451142  0.99925   ]] = 	 [5] 	 ['religon']
[[0.9840672  0.398261   0.13495126 0.3047344  0.58162874 0.37157065]] = 	 [0] 	 ['ST']
[[4.1538179e-02 9.9999940e-01 8.4075332e-04 3.2625526e-02 1.9854477e-01
  3.7974361e-01]] = 	 [1] 	 ['art']
[[1.08609855e-01 9.99999523e-01 7.72416592e-04 4.07330990e-02
  2.29737163e-01 4.18069184e-01]] = 	 [1] 	 ['art']
[[0.13954115 0.99994886 0.004195   0.09884748 0.22609967 0.27460098]] = 	 [1] 	 ['art']
[[0.21486339 0.99979585 0.01276469 0.19264224 0.25429073 0.2874838 ]] = 	 [1] 	 ['art']
[[3.8344234e-02 1.0000000e+00 8.5467822e-05 2.1513134e-02 1.5300035e-01
  2.2812390e-01]] = 	 [1] 	 ['art']
[[0.99999726 0.0994091  0.00100526 0.18401262 0.29807872 0.274534  ]] = 	 [0] 	 ['ST']
[[2.6279879e-01 2.8432786e-02 8.2998586e-06 7.72